# Kaggle Home Credit Default Risk - Simple LightGBM

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
import datetime as dt
import time
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [3]:
app_train = pd.read_csv('../data/application_train.csv')
print('application_train data shape: ', app_train.shape)

app_test = pd.read_csv('../data/application_test.csv')
print('application_test data shape: ', app_test.shape)

df_ssub = pd.read_csv('../data/sample_submission.csv')
print('sample_submission data shape: ', df_ssub.shape)

application_train data shape:  (307511, 122)
application_test data shape:  (48744, 121)
sample_submission data shape:  (48744, 2)


In [6]:
cat_features = []
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        cat_features.append(col)

X_train = app_train.drop(columns = ['SK_ID_CURR', 'TARGET']).copy()
X_test = app_test.drop(columns = ['SK_ID_CURR']).copy()
y_train = app_train['TARGET']

# Print out the new shapes
print('Training data shape: ', X_train.shape)
print('Testing data shape:  ', X_test.shape)

print('\nTraining LGBM model...')
ltrain = lgb.Dataset(X_train, label = y_train, feature_name=list(X_train.columns), categorical_feature=cat_features)

params = {}
params['metric'] = 'auc'
params['max_depth'] = 10
params['num_leaves'] = 132
params['min_data_in_leaf'] = 1000

# params['n_estimators'] = 10000    # replaced by num_boost_round
params['objective'] = 'binary'
params['class_weight'] = 'balanced'
params['reg_alpha'] = 0.1
params['reg_lambda'] = 0.1
params['subsample'] = 0.8

params['feature_fraction'] = .85
params['bagging_fraction'] = .95
params['bagging_freq'] = 8
params['learning_rate'] = 0.0025
params['verbosity'] = 0
params['n_jobs'] = -1

lgb_model = lgb.train(params, ltrain, verbose_eval=0, num_boost_round=10000)

print('Making predictions and praying for good results...')
y_pred = lgb_model.predict(X_test)

# app_test['TARGET'] = y_pred
# app_test['TARGET'] = app_test['TARGET'].apply(lambda x: abs(x))

# Make the submission dataframe
submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': y_pred})
submission['TARGET'] = submission['TARGET'].apply(lambda x: abs(x))

print('Saving predictions...')
# results = app_test[df_ssub.columns]
assert submission.shape == df_ssub.shape
submission.to_csv('../submissions/simple_lightgbm_{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, float_format='%.4f')
print('Done!')

Training data shape:  (307511, 120)
Testing data shape:   (48744, 120)

Training LGBM model...


C:\Users\WBirmingham\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1040: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


ValueError: DataFrame.dtypes for data must be int, float or bool. Did not expect the data types in fields NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, FONDKAPREMONT_MODE, HOUSETYPE_MODE, WALLSMATERIAL_MODE, EMERGENCYSTATE_MODE